In [1]:
import tensorflow as tf
print("TensorFlow version: ", tf.__version__)

TensorFlow version:  2.9.1


## Load a dataset

Load and prepare the MNIST dataset. Convert the sample data from integers to floating-point numbers

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11490434/11490434 [==============================] - 4s 0us/step


## Build a machine learning model

Build a `tf.keras.Sequential` model by stacking layers.

In [3]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10)
])

For each example, the model returns a vector of logits or log-odds scores, one for each class.

In [4]:
predictions = model(x_train[:1]).numpy()
predictions

array([[-0.1412586 ,  0.17658943,  0.09407218,  0.6268555 , -0.5381893 ,
         0.12391141,  0.4281706 , -0.30423665, -0.01014163, -0.11453978]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to probabilities for each class:

In [5]:
tf.nn.softmax(predictions).numpy()

array([[0.07964782, 0.10944943, 0.10078056, 0.17169656, 0.05355365,
        0.10383308, 0.14075823, 0.06766957, 0.09080657, 0.0818046 ]],
      dtype=float32)

Note: It is possible to bake the `tf.nn.softmax` function into the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to provide an exact and numerically stable loss calculation for all models when using a softmax output.

Define a loss function for training using `losses.SparseCategoricalCrossentropy`, which takes a vector of logits and a `True` index and returns a scalar loss for each example.

In [6]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class. The loss is zero if the model is sure of the correct class.

This untrained model gives proba